In [3]:
cd D:\LAB\ADR\AgenticAdr

D:\LAB\ADR\AgenticAdr


## Results

In [2]:
import pandas as pd

In [3]:
file_path = 'Code/AgenticAdr.xlsx'
df = pd.read_excel(file_path, sheet_name='Form Responses 2')
df.head()

,Timestamp,Name,Mail id,Repository URL,Years of industry experience.\n(Mention 0 years if no industry experience),Relevance (alignment with repository),Coherence (logical and clear structure),Completeness (coverage of key aspects),Conciseness (brevity without unnecessary detail),Overall,...,Completeness (coverage of key aspects).2,Conciseness (brevity without unnecessary detail) .2,Overall.2,Observations on generated ADRs.2,Relevance (alignment with repository).3,Coherence (logical and clear structure).3,Completeness (coverage of key aspects).3,Conciseness (brevity without unnecessary detail) .3,Overall.3,Observations on generated ADRs.3
0,2025-10-10 22:00:51.224,Hiya Bhatt,hiyabhatt2002@gmail.com,https://github.com/sa4s-serc/HarmonE,0,3,3,2,3,3,...,4,3,4,ADR-001: The generated ADR is correct. In the ...,5,4,5,5,5,ADR-001: ADR is correct. It just does not ment...
1,2025-10-13 11:18:54.486,Aneesh,sambu.aneesh@research.iiit.ac.in,https://github.com/sambuaneesh/why-py,0,5,4,3,4,4,...,4,5,5,"ok, this thing actually captured different und...",4,4,3,4,3,"brevity is okay, but this is too concise losin..."


In [4]:
df = df.rename(columns={
    'Years of industry experience.\n(Mention 0 years if no  industry experience)': 'Experience',
    'Relevance (alignment with repository)': 'LLM Gemini Relevance',
    'Coherence (logical and clear structure)': 'LLM Gemini Coherence',
    'Completeness (coverage of key aspects)': 'LLM Gemini Completeness',
    'Conciseness (brevity without unnecessary detail) ': 'LLM Gemini Conciseness',
    'Overall': 'LLM Gemini Overall',
    'Relevance (alignment with repository).1': 'LLM GPT-5 Relevance',
    'Coherence (logical and clear structure).1': 'LLM GPT-5 Coherence',
    'Completeness (coverage of key aspects).1': 'LLM GPT-5 Completeness',
    'Conciseness (brevity without unnecessary detail) .1': 'LLM GPT-5 Conciseness',
    'Overall.1': 'LLM GPT-5 Overall',
    'Relevance (alignment with repository).2': 'Agent Gemini Relevance',
    'Coherence (logical and clear structure).2': 'Agent Gemini Coherence',
    'Completeness (coverage of key aspects).2': 'Agent Gemini Completeness',
    'Conciseness (brevity without unnecessary detail) .2': 'Agent Gemini Conciseness',
    'Overall.2': 'Agent Gemini Overall',
    'Relevance (alignment with repository).3': 'Agent GPT-5 Relevance',
    'Coherence (logical and clear structure).3': 'Agent GPT-5 Coherence',
    'Completeness (coverage of key aspects).3': 'Agent GPT-5 Completeness',
    'Conciseness (brevity without unnecessary detail) .3': 'Agent GPT-5 Conciseness',
    'Overall.3': 'Agent GPT-5 Overall'
})

In [6]:
mean_series = df.mean(numeric_only=True)
split_index = mean_series.index.str.extract(r'(?P<Source>LLM|Agent)\s+(?P<Model>Gemini|GPT-5)\s+(?P<Metric>.+)')

mean_df = pd.DataFrame({
    'Value': mean_series.values
}, index=pd.MultiIndex.from_frame(split_index))

table = mean_df.reset_index().pivot_table(
    index=['Source', 'Model'],
    columns='Metric',
    values='Value'
).reset_index()

table = table[['Source', 'Model', 'Relevance', 'Coherence', 'Completeness', 'Conciseness', 'Overall']]

# table = table.drop('Metric', axis=1)
print(table.to_string(index=False))


Source  Model  Relevance  Coherence  Completeness  Conciseness  Overall
 Agent  GPT-5        4.5        4.0           4.0          4.5      4.0
 Agent Gemini        4.5        4.5           4.0          4.0      4.5
   LLM  GPT-5        3.0        3.5           3.0          3.5      3.0
   LLM Gemini        4.0        3.5           2.5          3.5      3.5


## Repo data

In [1]:
cd D:\LAB\ADR\AgenticAdr

D:\LAB\ADR\AgenticAdr


In [ ]:
import requests
import re
import json
from dotenv import load_dotenv
import os

load_dotenv()
GITHUB_KEY = os.getenv("GITHUB_AGENTIC_ADR")

In [3]:
def get_repo_languages(repo_url):
    """
    Fetches and prints the language statistics for a given GitHub repository URL.
    """
    
    # Use regex to extract 'owner/repo' from various GitHub URL formats
    # This handles URLs with or without '.git', trailing slashes, etc.
    match = re.search(r"github\.com/([^/]+)/([^/]+?)(?:\.git|/$|$)", repo_url)
    
    if not match:
        print(f"Error: Could not parse owner and repo from URL: {repo_url}")
        print("Expected format like: https://github.com/owner/repo")
        return

    owner = match.group(1)
    repo = match.group(2)
    
    # Construct the GitHub API URL for languages
    api_url = f"https://api.github.com/repos/{owner}/{repo}/languages"
    
    try:
        # Make the API request
        # # Adding a User-Agent is good practice for public APIs
        # headers = {"User-Agent": "language-checker-app"} 
        # response = requests.get(api_url, headers=headers)
        headers = {"Authorization": GITHUB_KEY}
        response = requests.get(api_url, headers=headers)
        
        # Raise an exception for bad status codes (like 404 Not Found or 403 Forbidden)
        response.raise_for_status() 
        
        # Get the JSON data (this will be a dictionary like {"Python": 12345, "TeX": 6789})
        languages_data = response.json()
        
        if not languages_data:
            print(f"No language data found for {owner}/{repo}.")
            print("The repository might be empty or GitHub hasn't processed it yet.")
            return

        # Calculate the total bytes of code
        total_bytes = sum(languages_data.values())
        
        language_percentages = {} # Create an empty dictionary
        
        # Calculate and store the percentage for each language
        for language, bytes_count in languages_data.items():
            percentage = (bytes_count / total_bytes) * 100
            language_percentages[language] = round(percentage, 1) # Add to dict, rounded
            
        # Sort the dictionary by percentage (descending) for cleaner output
        sorted_languages = dict(sorted(
            language_percentages.items(), 
            key=lambda item: item[1], 
            reverse=True
        ))
        sorted_languages['total_bytes'] = total_bytes

        # Convert the final dictionary to a JSON string and return it
        # 'indent=2' makes the JSON output pretty and readable
        return json.dumps(sorted_languages, indent=2)

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            print(f"Error: Repository not found. Check the URL: {repo_url}")
        elif response.status_code == 403:
            print(f"Error: API rate limit exceeded or access forbidden.")
            print("GitHub's public API has a rate limit. Please wait a while before trying again.")
        else:
            print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred during the request (e.g., network issue): {err}")



In [10]:
repos = [
    "https://github.com/sa4s-serc/HarmonE",
    "https://github.com/sambuaneesh/why-py",
    "https://github.com/stanfordnlp/dspy",
    "https://github.com/ameykaran/esell/",
    "https://github.com/miryala10sathvika/Food-delivery",
    "https://github.com/legend479/HARPP",
    "https://github.com/miryala10sathvika/Drawingeditor",
    "https://github.com/sudokara/monkeytpe-contest",
    "https://github.com/likhithkanigolla/CAPS-IIITH",
    "https://github.com/pgexporter/pgexporter",
    "https://github.com/sudokara/SpecFlow",
    "https://github.com/adyanshkakran/need-for-speed",
    "https://github.com/adyanshkakran/MLOps",
    "https://github.com/divyanash911/RSS-Reader",
    "https://github.com/divyanash911/design-for-social-innovation-project",
    "https://github.com/yashaswinee/QuizAppMicroservices/tree/microservices",
    "https://github.com/sa4s-serc/EdgeMLBalancer",
    "https://github.com/sa4s-serc/ecomls",
    "https://github.com/sa4s-serc/switch",
    # "https://gitlab.com/20a_akhila/amazon-clone",
    # "https://gitlab.com/20a_akhila/movie-database",
    # "https://gitlab.com/20a_akhila/silly_story_generator",
    "https://github.com/OSDG-IIITH/CabMiloge",
    "https://github.com/ShailenderGoyal/Enigmatrix_Salesforce_Hackathon",
    "https://github.com/ShailenderGoyal/PageRank-Algorithm",
    "https://github.com/adyanshkakran/Reddit-clone",
    "https://github.com/adyanshkakran/RL_project",
    "https://github.com/adyanshkakran/Shravana"
]

In [ ]:
all_repos_data = {}

for repo in repos:
    json_output = get_repo_languages(repo)
    all_repos_data[repo] = json_output

with open("Code/all_repos_languages.json", "w", encoding="utf-8") as f:
    json.dump(all_repos_data, f, indent=2)

No language data found for divyanash911/RSS-Reader.
The repository might be empty or GitHub hasn't processed it yet.
Error: Could not parse owner and repo from URL: https://github.com/yashaswinee/QuizAppMicroservices/tree/microservices
Expected format like: https://github.com/owner/repo


## Analysis

In [1]:
import pandas as pd
import json

In [ ]:
with open("Code/all_repos_languages.json", "r") as f:
    data = json.load(f)

lang_df = pd.Series(data).apply(pd.Series)
lang_df = pd.DataFrame.from_dict(data, orient='index', columns=['languages'])
lang_df = lang_df.reset_index()

lang_df = lang_df.rename(columns={'index': 'Repository URL'})
lang_df.head()

data

{'https://github.com/sa4s-serc/HarmonE': '{\n  "Python": 94.3,\n  "Shell": 5.7,\n  "total_bytes": 44320\n}',
 'https://github.com/sambuaneesh/why-py': '{\n  "Python": 79.4,\n  "TeX": 20.5,\n  "Makefile": 0.1,\n  "total_bytes": 111666\n}',
 'https://github.com/stanfordnlp/dspy': '{\n  "Python": 99.6,\n  "JavaScript": 0.4,\n  "total_bytes": 1517449\n}',
 'https://github.com/ameykaran/esell/': '{\n  "TypeScript": 42.8,\n  "Python": 42.8,\n  "HTML": 12.9,\n  "CSS": 1.0,\n  "Shell": 0.4,\n  "JavaScript": 0.2,\n  "total_bytes": 315716\n}',
 'https://github.com/miryala10sathvika/Food-delivery': '{\n  "JavaScript": 65.8,\n  "HTML": 17.1,\n  "CSS": 17.1,\n  "total_bytes": 95606\n}',
 'https://github.com/legend479/HARPP': '{\n  "Python": 100.0,\n  "total_bytes": 41531\n}',
 'https://github.com/miryala10sathvika/Drawingeditor': '{\n  "Python": 100.0,\n  "total_bytes": 39818\n}',
 'https://github.com/sudokara/monkeytpe-contest': '{\n  "JavaScript": 69.0,\n  "HTML": 31.0,\n  "total_bytes": 22879\n}

In [17]:
df = pd.read_excel("code/AgenticAdr.xlsx", sheet_name='Form Responses 2')

merged_df = pd.merge(df, lang_df, 
                     left_on='Repository URL', 
                     right_on='Repository URL', 
                     how='left')
merged_df.head()

,Timestamp,Name,Mail id,Repository URL,Years of industry experience.\n(Mention 0 years if no industry experience),Relevance (alignment with repository),Coherence (logical and clear structure),Completeness (coverage of key aspects),Conciseness (brevity without unnecessary detail),Overall,...,Conciseness (brevity without unnecessary detail) .2,Overall.2,Observations on generated ADRs.2,Relevance (alignment with repository).3,Coherence (logical and clear structure).3,Completeness (coverage of key aspects).3,Conciseness (brevity without unnecessary detail) .3,Overall.3,Observations on generated ADRs.3,languages
0,2025-10-10 22:00:51.224,Hiya Bhatt,hiyabhatt2002@gmail.com,https://github.com/sa4s-serc/HarmonE,0.0,3,3,2,3,3,...,3,4,ADR-001: The generated ADR is correct. In the ...,5,4,5,5,5,ADR-001: ADR is correct. It just does not ment...,"{\n ""Python"": 94.3,\n ""Shell"": 5.7,\n ""tota..."
1,2025-10-13 11:18:54.486,Aneesh,sambu.aneesh@research.iiit.ac.in,https://github.com/sambuaneesh/why-py,0.0,5,4,3,4,4,...,5,5,"ok, this thing actually captured different und...",4,4,3,4,3,"brevity is okay, but this is too concise losin...","{\n ""Python"": 79.4,\n ""TeX"": 20.5,\n ""Makef..."
2,2025-10-14 16:38:21.523,Sreemaee Akshathala,sreemaee.akshathala@research.iiit.ac.in,https://github.com/stanfordnlp/dspy,0.0,4,4,5,3,4,...,3,3,The consequences are not divided into positive...,4,4,2,2,3,Too brief. The context basically lists the cla...,"{\n ""Python"": 99.6,\n ""JavaScript"": 0.4,\n ..."
3,2025-10-14 19:33:47.338,Amey Karan,amey.karan@research.iiit.ac.in,github.com/ameykaran/esell/,0.0,5,4,3,4,4,...,5,4,This method is the best till now. It clearly c...,5,4,4,3,3,This method produces too many ADRs. Most of th...,NaN
4,2025-10-14 23:20:40.417,Miryala Sathvika,miryala.sathvika@research.iiit.ac.in,https://github.com/miryala10sathvika/Food-deli...,0.0,5,5,3,4,4,...,4,4,"here, the format is a bit different from the u...",5,4,5,4,5,the good thing here is it is trying to capture...,"{\n ""JavaScript"": 65.8,\n ""HTML"": 17.1,\n ""..."


In [18]:
merged_df.to_excel("Code/merged_data.xlsx", index=False)

## Line count

In [12]:
import tempfile
import subprocess
import os

In [17]:
def count_lines(repo_url):
    with tempfile.TemporaryDirectory() as tmpdir:
        # print(f"Cloning {repo_url} into {tmpdir}...")
        subprocess.run(["git", "clone", "--depth", "1", repo_url, tmpdir], check=True)

        total_lines = 0
        for root, _, files in os.walk(tmpdir):
            for f in files:
                try:
                    with open(os.path.join(root, f), 'r', errors='ignore') as file:
                        total_lines += sum(1 for _ in file)
                except Exception:
                    pass

        return total_lines

In [ ]:
repos = [
    "https://github.com/sa4s-serc/HarmonE",
    "https://github.com/sambuaneesh/why-py",
    "https://github.com/stanfordnlp/dspy",
    "https://github.com/ameykaran/esell/",
    "https://github.com/miryala10sathvika/Food-delivery",
    "https://github.com/legend479/HARPP",
    "https://github.com/miryala10sathvika/Drawingeditor",
    "https://github.com/sudokara/monkeytpe-contest",
    "https://github.com/likhithkanigolla/CAPS-IIITH",
    "https://github.com/pgexporter/pgexporter",
    "https://github.com/sudokara/SpecFlow",
    "https://github.com/adyanshkakran/need-for-speed",
    "https://github.com/adyanshkakran/MLOps",
    "https://github.com/divyanash911/RSS-Reader",
    "https://github.com/divyanash911/design-for-social-innovation-project",
    "https://github.com/yashaswinee/QuizAppMicroservices/tree/microservices",
    "https://github.com/sa4s-serc/EdgeMLBalancer",
    "https://github.com/sa4s-serc/ecomls",
    "https://github.com/sa4s-serc/switch",
    "https://gitlab.com/20a_akhila/amazon-clone",
    "https://gitlab.com/20a_akhila/movie-database",
    "https://gitlab.com/20a_akhila/silly_story_generator",
    "https://github.com/OSDG-IIITH/CabMiloge",
    "https://github.com/ShailenderGoyal/Enigmatrix_Salesforce_Hackathon",
    "https://github.com/ShailenderGoyal/PageRank-Algorithm",
    "https://github.com/adyanshkakran/Reddit-clone",
    "https://github.com/adyanshkakran/RL_project",
    "https://github.com/adyanshkakran/Shravana"
]

lines = {}

for repo in repos:
    lines[repo] = count_lines(repo)


In [24]:
values = lines.values()

min_val = min(values)
max_val = max(values)
avg_val = sum(values) / len(values)

print(f"Min: {min_val}")
print(f"Max: {max_val}")
print(f"Avg: {avg_val:.2f}")

Min: 1008
Max: 5559938
Avg: 350500.41
